<a href="https://colab.research.google.com/github/grid-rider/NYU_Independent_Study_Math_PUBLIC/blob/main/Math_Independent_Study_Complexity_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install torch datasets performer_pytorch matplotlib xformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import math
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import psutil
try:
    import xformers.ops as xops
    HAS_XFORMERS = True
except ImportError:
    HAS_XFORMERS = False
    print("xFormers not installed!!!!!.")

# lucidrains Performer, for approximate O(n) attention (explicitly: Performer method). USING his version as a complete implementation is very complex and inefficient.
try:
    from performer_pytorch import Performer
    HAS_PERFORMER = True
except ImportError:
    HAS_PERFORMER = False
    print("lucidrains performer-pytorch not installed. pip install performer-pytorch if you want the Performer block.")

###############################################################################
#                      Attention Blocks: xFormers, Performer, Nystrom
###############################################################################
class XFormersAttentionBlock(nn.Module):
    def __init__(self, embed_dim=128, num_heads=4):
        super().__init__()
        if not HAS_XFORMERS:
            raise RuntimeError("xFormers not installed.")
        assert embed_dim % num_heads == 0
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads

        self.q_proj = nn.Linear(embed_dim, embed_dim)
        self.k_proj = nn.Linear(embed_dim, embed_dim)
        self.v_proj = nn.Linear(embed_dim, embed_dim)
        self.out_proj = nn.Linear(embed_dim, embed_dim)

        self.ln1 = nn.LayerNorm(embed_dim)
        self.ff = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * 4),
            nn.ReLU(),
            nn.Linear(embed_dim * 4, embed_dim)
        )
        self.ln2 = nn.LayerNorm(embed_dim)

    def forward(self, x):
        B, T, E = x.shape
        q = self.q_proj(x).view(B, T, self.num_heads, self.head_dim)
        k = self.k_proj(x).view(B, T, self.num_heads, self.head_dim)
        v = self.v_proj(x).view(B, T, self.num_heads, self.head_dim)

        q = q.permute(0,2,1,3).reshape(B*self.num_heads, T, self.head_dim)
        k = k.permute(0,2,1,3).reshape(B*self.num_heads, T, self.head_dim)
        v = v.permute(0,2,1,3).reshape(B*self.num_heads, T, self.head_dim)

        # Memory-efficient attention from xFormers (baseline full quadratic attention)
        attn_out = xops.memory_efficient_attention(q, k, v, attn_bias=None)
        attn_out = attn_out.view(B, self.num_heads, T, self.head_dim).permute(0,2,1,3).contiguous()
        attn_out = attn_out.view(B, T, E)

        x = x + attn_out
        x = self.ln1(x)
        ff_out = self.ff(x)
        x = x + ff_out
        x = self.ln2(x)
        return x

class PerformerAttentionBlock(nn.Module):
    def __init__(self, embed_dim=128, num_heads=4, nb_features=64):
        super().__init__()
        if not HAS_PERFORMER:
            raise RuntimeError("performer-pytorch not installed.")
        self.performer = Performer(
            dim=embed_dim,
            depth=1,
            heads=num_heads,
            dim_head=embed_dim // num_heads,
            nb_features=nb_features,  # explicitly the Performer method
            causal=False
        )
        self.ln1 = nn.LayerNorm(embed_dim)
        self.ff = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * 4),
            nn.ReLU(),
            nn.Linear(embed_dim * 4, embed_dim)
        )
        self.ln2 = nn.LayerNorm(embed_dim)

    def forward(self, x):
        attn_out = self.performer(x)  # Performer (orthogonal random features)
        x = x + attn_out
        x = self.ln1(x)
        ff_out = self.ff(x)
        x = x + ff_out
        x = self.ln2(x)
        return x

class NystromAttentionBlock(nn.Module):
    def __init__(self, embed_dim=128, num_heads=4, seq_len=512, num_landmarks=64):
        super().__init__()
        assert embed_dim % num_heads == 0
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.num_landmarks = num_landmarks
        self.seq_len = seq_len

        self.q_proj = nn.Linear(embed_dim, embed_dim)
        self.k_proj = nn.Linear(embed_dim, embed_dim)
        self.v_proj = nn.Linear(embed_dim, embed_dim)
        self.out_proj = nn.Linear(embed_dim, embed_dim)

        self.ln1 = nn.LayerNorm(embed_dim)
        self.ff = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * 4),
            nn.ReLU(),
            nn.Linear(embed_dim * 4, embed_dim)
        )
        self.ln2 = nn.LayerNorm(embed_dim)

    def iterative_inv(self, softmax_mat: torch.Tensor, n_iter=6):
        # Simple iterative inversion routine (as provided in your code)
        softmax_mat = torch.clamp(softmax_mat, min=1e-9)
        I = torch.eye(softmax_mat.size(-1), device=softmax_mat.device, dtype=softmax_mat.dtype)
        K = softmax_mat
        row_sums = torch.sum(K, dim=-2)
        max_per_batch = torch.max(row_sums, dim=-1).values
        max_per_batch = torch.clamp(max_per_batch, min=1e-9)
        max_per_batch = max_per_batch.unsqueeze(-1).unsqueeze(-1)
        V = (1.0 / max_per_batch) * K.transpose(-1, -2)
        for _ in range(n_iter):
            KV = torch.matmul(K, V)
            t1 = 7 * I - KV
            t2 = torch.matmul(KV, t1)
            t3 = 15 * I - t2
            t4 = torch.matmul(KV, t3)
            t5 = 13 * I - t4
            V = 0.25 * torch.matmul(V, t5)
        return V

    def forward(self, x):
        B, T, E = x.shape
        H = self.num_heads
        D = self.head_dim

        Q = self.q_proj(x).view(B, T, H, D).transpose(1, 2)
        K = self.k_proj(x).view(B, T, H, D).transpose(1, 2)
        V = self.v_proj(x).view(B, T, H, D).transpose(1, 2)

        r = min(self.num_landmarks, T)
        seg_size = math.ceil(T / r)
        Q_lands, K_lands = [], []
        # Compute landmark summaries by averaging segments
        for i in range(r):
            start = i * seg_size
            end = min((i + 1) * seg_size, T)
            Q_seg = Q[:, :, start:end, :].mean(dim=2, keepdim=True)
            K_seg = K[:, :, start:end, :].mean(dim=2, keepdim=True)
            Q_lands.append(Q_seg)
            K_lands.append(K_seg)
        Q_lands = torch.cat(Q_lands, dim=2)
        K_lands = torch.cat(K_lands, dim=2)

        scale = 1. / math.sqrt(D + 1e-9)
        A = torch.matmul(Q * scale, K_lands.transpose(-1, -2))
        A = F.softmax(A, dim=-1)
        A = torch.clamp(A, min=1e-9)

        B_mat = torch.matmul(Q_lands * scale, K_lands.transpose(-1, -2))
        B_mat = F.softmax(B_mat, dim=-1)
        B_mat = torch.clamp(B_mat, min=1e-9)

        C_ = torch.matmul(Q_lands * scale, K.transpose(-1, -2))
        C_ = F.softmax(C_, dim=-1)
        C_ = torch.clamp(C_, min=1e-9)
        C_ = torch.matmul(C_, V)

        B_inv = self.iterative_inv(B_mat, n_iter=6)
        BC = torch.matmul(B_inv, C_)
        X_hat = torch.matmul(A, BC)
        X_hat = X_hat.transpose(1, 2).contiguous().view(B, T, E)
        out = self.out_proj(X_hat)

        x = x + out
        x = self.ln1(x)
        ff_out = self.ff(x)
        x = x + ff_out
        x = self.ln2(x)
        return x

###############################################################################
#                       MY Model Wrapper for Attention Experiments
###############################################################################
class AttentionTestModel(nn.Module):
    def __init__(self, embed_dim=256, num_heads=4, seq_len=512,
                 attn_type="xformers", nb_features=64, num_landmarks=64):
        super().__init__()
        self.embed_dim = embed_dim
        self.seq_len = seq_len
        self.embedding = nn.Embedding(10000, embed_dim)  # arbitrary vocab size

        if attn_type == "xformers":
            self.attn_block = XFormersAttentionBlock(embed_dim, num_heads)
        elif attn_type == "performer":
            self.attn_block = PerformerAttentionBlock(embed_dim, num_heads, nb_features=nb_features)
        elif attn_type == "nystrom":
            self.attn_block = NystromAttentionBlock(embed_dim, num_heads, seq_len=seq_len, num_landmarks=num_landmarks)
        else:
            raise ValueError(f"Unknown attn_type: {attn_type}")

        self.ffn = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * 4),
            nn.ReLU(),
            nn.Linear(embed_dim * 4, embed_dim)
        )

    def forward(self, x):
        # x: (B, T)
        x = self.embedding(x)
        x = self.attn_block(x)
        x = self.ffn(x)
        return x

###############################################################################
#                    Benchmarking Over Sequence Lengths and Logging Resources
###############################################################################
def benchmark_forward_pass(model, seq_len=512, batch_size=8, vocab_size=10000, steps=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device).eval()
    dummy_input = torch.randint(0, vocab_size, (batch_size, seq_len), device=device)
    times = []
    with torch.no_grad():
        for _ in range(steps):
            start = time.time()
            _ = model(dummy_input)
            if device.type == 'cuda':
                torch.cuda.synchronize()
            end = time.time()
            times.append(end - start)
    return sum(times) / len(times)

def log_system_resources():
    cpu_usage = psutil.cpu_percent(interval=1)
    mem = psutil.virtual_memory()
    mem_usage = mem.percent
    log_str = f"CPU Usage: {cpu_usage:.1f}%, Memory Usage: {mem_usage:.1f}%"
    # If GPU is available, also log GPU memory allocated
    if torch.cuda.is_available():
        gpu_mem_alloc = torch.cuda.memory_allocated() / (1024**2)
        gpu_mem_reserved = torch.cuda.memory_reserved() / (1024**2)
        log_str += f", GPU Memory Allocated: {gpu_mem_alloc:.1f} MB, Reserved: {gpu_mem_reserved:.1f} MB"
    return log_str

def run_attention_benchmarks(seq_lengths, attn_types, embed_dim=256, batch_size=8, steps=50):
    results = {attn: [] for attn in attn_types}
    resource_logs = {attn: [] for attn in attn_types}
    vocab_size = 10000

    for seq_len in seq_lengths:
        print(f"\nBenchmarking sequence length: {seq_len}")
        for attn in attn_types:
            print(f"  -> Method: {attn}")
            model = AttentionTestModel(embed_dim=embed_dim,
                                       seq_len=seq_len,
                                       attn_type=attn,
                                       nb_features=64,
                                       num_landmarks=64)
            fwd_time = benchmark_forward_pass(model, seq_len=seq_len, batch_size=batch_size,
                                              vocab_size=vocab_size, steps=steps)
            results[attn].append(fwd_time)
            res_log = log_system_resources()
            resource_logs[attn].append(res_log)
            print(f"     Average forward pass time: {fwd_time:.4f} sec")
            print(f"     System Resources: {res_log}")
    return results, resource_logs

#Used same function with minor adaptation from modeling_sin.py as I wanted to keeep the minimal look
def plot_attention_benchmarks(seq_lengths, results):
    plt.figure(figsize=(14, 12))
    colors = {
        "xformers": "blue",
        "performer": "magenta",
        "nystrom": "green"
    }
    linewidths = {
        "xformers": 2,
        "performer": 1.5,
        "nystrom": 2
    }
    markers = {
        "xformers": "o",
        "performer": "s",
        "nystrom": "D"
    }
    for attn, times in results.items():
        plt.plot(seq_lengths, times, marker=markers.get(attn, "o"),
                 color=colors.get(attn, "black"), linewidth=linewidths.get(attn, 2),
                 label=attn.capitalize())

    plt.xlabel("Sequence Length", fontsize=14)
    plt.ylabel("Average Forward Pass Time (sec)", fontsize=14)
    plt.title("Attention Forward Pass Time vs. Sequence Length", fontsize=16)
    plt.legend(fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig("attention_benchmarks.png", dpi=300)
    plt.close()
    print("Plot 'attention_benchmarks.png' saved.\n")


###############################################################################
#                         Main Experiment for Section 4 Of My paper
###############################################################################
def main():
    seq_lengths = [256, 512, 1024, 2048, 4096, 8192, 16384] # You could technically added to this further
    attn_types = ["xformers", "performer", "nystrom"]
    print("Running extreme attention mechanism benchmarks over varying sequence lengths...\n")
    results, resource_logs = run_attention_benchmarks(seq_lengths, attn_types, embed_dim=256, batch_size=8, steps=50)
    plot_attention_benchmarks(seq_lengths, results)
    print("\nDetailed Summary of Average Forward Pass Times and Resource Logs:")
    for attn in attn_types:
        print(f"\nMethod: {attn}")
        for idx, seq_len in enumerate(seq_lengths):
            time_val = results[attn][idx]
            res_log = resource_logs[attn][idx]
            print(f"  Seq Length {seq_len:5d}: Time = {time_val:.4f} sec | {res_log}")

if __name__ == "__main__":
    main()


Running extreme attention mechanism benchmarks over varying sequence lengths...


Benchmarking sequence length: 256
  -> Method: xformers
     Average forward pass time: 0.0035 sec
     System Resources: CPU Usage: 7.0%, Memory Usage: 16.5%, GPU Memory Allocated: 22.9 MB, Reserved: 2506.0 MB
  -> Method: performer
     Average forward pass time: 0.0044 sec
     System Resources: CPU Usage: 2.5%, Memory Usage: 16.5%, GPU Memory Allocated: 24.9 MB, Reserved: 2506.0 MB
  -> Method: nystrom
     Average forward pass time: 0.0059 sec
     System Resources: CPU Usage: 2.0%, Memory Usage: 16.5%, GPU Memory Allocated: 22.9 MB, Reserved: 2506.0 MB

Benchmarking sequence length: 512
  -> Method: xformers
     Average forward pass time: 0.0044 sec
     System Resources: CPU Usage: 2.0%, Memory Usage: 16.5%, GPU Memory Allocated: 22.9 MB, Reserved: 2506.0 MB
  -> Method: performer
     Average forward pass time: 0.0064 sec
     System Resources: CPU Usage: 2.0%, Memory Usage: 16.5%, GPU Memory All